In [2]:
import os
import random as rnd
import pandas as pd 

import pgutils as pg
import pgmath as pm
import algorithm as al
import numpy as np

In [3]:
root_dir = r'C:\Users\eleon\PycharmProjects\SM_Exam\data\yes_small'
#root_dir = './data/yes_small'

In [4]:
#test_data=[]
with open(os.path.join(root_dir, 'test.txt')) as f:
#with open(root_dir + "\\test.txt", "r") as f:

    test_data = f.readlines()
    
test_dataset = pg.data_to_list(test_data[2:])

In [5]:
#train_data=[]
with open(os.path.join(root_dir,"train.txt"), "r") as f:
    train_data = f.readlines()
    
#train_dataset = pg.data_to_list(train_data[2:])

In [6]:
song_hash = pd.read_csv(os.path.join(root_dir,"song_hash.txt"), sep="\t", header=None)


In [7]:
# setting parameters
dimension = 2#rnd.choice([2, 5,10,25,50,100])
# regularization parameter set by cross validation
lam = rnd.choice([0.0001, 0.001, 0.01, 0.1, 1, 10, 20, 50, 100, 500, 1000])
# regularization parameter for dual point model
nu = rnd.choice([0, lam])
# threshold for landmark dimension
r = 0.2
# number of landmark
n_landmarks = 3#50
# num iteration 100 o 200
n_iter = 60 #rnd.choice([100, 200])
# tau predefined learning rate
tau = 0.5

In [8]:
toy_dataset=[[0,6,3,2,1,4,2],[2,0,6,5,6,3,0],[0,5,2,1,6],[5,6,2],[6,5,2,3,5,6,2]]

songs = 7
transition_matrix = pg.transition_count(songs, toy_dataset)
position = np.random.rand(songs, dimension)
num_transition = np.sum(transition_matrix)
params = pm.AlgParams(lam, nu, tau, num_transition, n_landmarks, r, dimension, n_iter)

In [9]:
X = al.single_point_algorithm(songs, transition_matrix, params)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26


In [10]:
dummy_landmarks = [[i for i in range(songs)] for j in range(songs)]
d2 = pm.Distances.delta(X)
prob_matrix = np.exp(-d2)/pm.Distances.zeta(d2,X,dummy_landmarks)
cum_sum = np.sum(prob_matrix, axis=1)
prob_matrix = prob_matrix / cum_sum

In [11]:
print(prob_matrix)
print(np.sum(prob_matrix, axis=1))

[[0.16574914 0.14384048 0.14884891 0.13562863 0.14424961 0.14099463
  0.1349401 ]
 [0.12643577 0.18856559 0.1157633  0.13465182 0.11941971 0.11501597
  0.13797652]
 [0.14570234 0.12891484 0.16932864 0.12855955 0.14501758 0.15791592
  0.12752357]
 [0.13805281 0.15592555 0.13368336 0.16283861 0.13985282 0.13773493
  0.1615264 ]
 [0.14915748 0.140481   0.15318993 0.14207176 0.16029533 0.15338883
  0.14092887]
 [0.12890582 0.11962978 0.14749423 0.12371438 0.13562299 0.18129311
  0.12304612]
 [0.13558749 0.15772298 0.13090258 0.15945136 0.13694562 0.13523109
  0.16495773]]
[1.0142515  0.93782868 1.00296244 1.02961449 1.03951321 0.95970644
 1.02079887]


In [12]:
toy_test=[[0,4,5,3,6],[4,3,2,6],[0,2,1,0,5]]
# evaluate test performance using the average log-likelihood as metric
# it is defined as log(Pr(d_test))/n_test)
# n_test number of transition in the test set
songs = 7 #len(song_hash)
n_test = np.sum(pg.transition_count(songs,toy_test))
print(n_test)

11.0


In [14]:
import math as mt
def log_like(test_set, probability_matrix):
    count=0
    for i in range(len(test_set)):
        for predecessor, successor in zip(test_set[i][:-1], test_set[i][1:]):
            count = count + mt.log(probability_matrix[predecessor, successor])
            
    return count
    
    
evaluation = log_like(toy_test,prob_matrix) /n_test
print(evaluation)
#result = sum( sum(dimension.D[s, i.index()] ) for i in range(len(test_dataset)))  

-1.975226823881699
